# Udacity MLND Capstone — Tweet Classifier

## Prep
* load libraries
* establish baseline algorithm (Multinomial Naive Bayes)

In [12]:
#import nltk
#nltk.download()

In [13]:
import numpy as np
import pandas as pd

from keras.models import Sequential
from keras.layers import Dense, LSTM
from keras.layers.convolutional import Conv1D, MaxPooling1D
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence

In [14]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

tweets = pd.read_csv("./ExtractedTweets.csv")

---

## Baseline Algorithm (Multinomial NB)

In [15]:

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
vectorizer = CountVectorizer()
X_train_counts = vectorizer.fit_transform(tweets.Tweet)

print(X_train_counts.shape)

(86460, 126330)


In [16]:

tweet_accounts = tweets.iloc[:, :2].drop_duplicates()
accounts_train, accounts_test = train_test_split(tweet_accounts.Handle, stratify=tweet_accounts.Party, \
                                                 test_size=0.2, random_state=41)

tweets_train = tweets[tweets.Handle.isin(accounts_train)].reset_index().drop('index', axis=1)
tweets_test = tweets[tweets.Handle.isin(accounts_test)].reset_index().drop('index', axis=1)

In [17]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

mnb_pipeline = Pipeline([('vec', CountVectorizer()), ('tfidf', TfidfTransformer()), ('clf', MultinomialNB())])

mnb_pipeline.fit(tweets_train.Tweet, tweets_train.Party)

p = mnb_pipeline.predict(tweets_test.Tweet)
np.mean(p == tweets_test.Party)

0.75217089267106629

---

## Neural Network

In [21]:
from nltk.tokenize.casual import TweetTokenizer
from nltk import FreqDist

tokenizer = TweetTokenizer(reduce_len=True)

top_words = 20000

tweets_nn = tweets
tweets_nn.Tweet = tweets_nn.Tweet.apply(tokenizer.tokenize)

In [24]:
print("...")

fdist = FreqDist(word for tweet in tweets_nn.Tweet for word in tweet)
print("...fdist")
terms = [term for term, count in fdist.most_common(top_words)]
print("...terms")
tweets_nn.Tweet = tweets_nn.Tweet.apply(\
                    lambda tweet:[terms.index(term)\
                    if term in terms else 0 for term in tweet]
                )
print("...term lambda")

nn_tweeters = tweets_nn.iloc[:,:2].drop_duplicates()

print("^^^")

...
...fdist
...terms
...term lambda
^^^


In [55]:

nn_tweeters_train, nn_tweeters_test = train_test_split(nn_tweeters.Handle,\
                           stratify=nn_tweeters.Party, test_size=0.2, random_state=42)

nn_tweets_train = tweets_nn[tweets_nn.Handle.isin(nn_tweeters_train)].reset_index().drop('index', axis=1)
nn_tweets_test = tweets_nn[tweets_nn.Handle.isin(nn_tweeters_test)].reset_index().drop('index',axis=1)


In [58]:
print(nn_tweets_train.shape)
print(nn_tweets_test.shape)
nn_tweets_test.head()

(69185, 3)
(17275, 3)
----


,Party,Handle,Tweet
0,Democrat,RepJackyRosen,"[383, 3, 24, 480, 15, 19, 0, 4, 0, 0, 0, 16, 6..."
1,Democrat,RepJackyRosen,"[25, 1615, 2138, 5480, 470, 4, 119, 108, 7, 69..."
2,Democrat,RepJackyRosen,"[168, 275, 3, 0, 6369, 10, 40, 0, 1672, 150, 0..."
3,Democrat,RepJackyRosen,"[3852, 5575, 3, 42, 0, 104, 41, 261, 24, 5580,..."
4,Democrat,RepJackyRosen,"[52, 22, 4, 4843, 4, 10, 1627, 80, 3, 24, 110,..."


In [59]:
nn_tweets_train.head()


,Party,Handle,Tweet
0,Democrat,RepDarrenSoto,"[51, 4, 221, 1205, 161, 3, 3314, 2, 191, 3, 77..."
1,Democrat,RepDarrenSoto,"[11, 0, 6, 4354, 5971, 3406, 60, 0, 6931, 1677..."
2,Democrat,RepDarrenSoto,"[11, 12095, 6, 2, 1967, 11171, 21, 1222, 3008,..."
3,Democrat,RepDarrenSoto,"[11, 0, 6, 1352, 17, 1967, 2, 87, 10, 381, 0, ..."
4,Democrat,RepDarrenSoto,"[11, 0, 6, 1222, 1079, 1810, 13, 2564, 752, 45..."


In [60]:
np.random.seed(42)

X_train = np.array(nn_tweets_train.Tweet)
X_test = np.array(nn_tweets_test.Tweet)
y_train = np.array((nn_tweets_train.Party == 'Democrat').astype(int))
y_test = np.array((nn_tweets_test.Party == 'Democrat').astype(int))

In [61]:
tweet_length = 60
X_train = sequence.pad_sequences(X_train, maxlen=tweet_length)
X_test = sequence.pad_sequences(X_test, maxlen=tweet_length)

In [65]:
embedding_vector_length = 32
model = Sequential()
model.add(Embedding(top_words, embedding_vector_length, input_length=tweet_length))
model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy',])
print(model.summary())
model.fit(X_train, y_train, epochs=3, batch_size=64)
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 60, 32)            480000    
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 60, 32)            3104      
_________________________________________________________________
max_pooling1d_4 (MaxPooling1 (None, 30, 32)            0         
_________________________________________________________________
lstm_4 (LSTM)                (None, 100)               53200     
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 101       
Total params: 536,405
Trainable params: 536,405
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/3
69185/69185 [==============================] - 48s - loss: 0.4808 - acc: 0.7482    
Epoch 2/3
69185/69185 [==============